In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.multivariate.manova import MANOVA
from sklearn.preprocessing import StandardScaler

En el pair programming de hoy usaremos el set de datos que guardastéis en el pair programming de normalización y estandarización.

Hasta ahora habéis estado evaluando las características de vuestro set de datos y habéis hecho una gran exploración, es el momento de hacer vuestro primer ANOVA! En el ejercicio de hoy tendréis que hacer un ANOVA con vuestro datos y hacer una interpretación de los resultados.

📌 NOTA Puede que vuestros datos no se ajusten o no cumplan todas las asunciones, no pasa nada, haced el ANOVA e interpretad los resultados. En próximas lecciones aprenderemos que podemos hacer cuando nos encontramos en esta situación.

In [2]:
df = pd.read_csv("Datos/possum_esta.csv", index_col = 0)
df.head()

,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,age_norm,age_log,age_raiz,age_Sklearn,belly_esta
case,,,,,,,,,,,,,,,,,,
1,1,Vic,m,8.0,-0.028932,60.4,89.0,36.0,74.5,54.5,15.2,0.736833,0.150424,0.522837,2.079442,2.828427,0.625,1.235889
2,1,Vic,f,6.0,0.392875,57.6,91.5,36.5,72.5,51.2,16.0,1.473667,0.514241,0.272837,1.791759,2.449490,0.625,0.149699
3,1,Vic,f,6.0,0.167912,60.0,95.5,39.0,75.4,51.9,15.5,0.491222,0.514241,0.272837,1.791759,2.449490,0.625,0.511762
4,1,Vic,f,6.0,-0.310136,57.1,92.0,38.0,76.1,52.2,15.2,0.736833,0.150424,0.272837,1.791759,2.449490,0.125,0.511762
5,1,Vic,f,2.0,0.139791,56.3,85.5,36.0,71.0,53.2,15.1,1.473667,-0.213392,-0.227163,0.693147,1.414214,0.000,0.149699


Vamos a iniciar nuestro anova con la variable respuesta age_raiz (la normalización que mejor nos funcionó) y con las varaibles predictoras que habíamos escogido en su momento por tener una mayor correlación con age. Estas eran "hdlngth", "chest" y "belly.

Tras los ejercicios anteriores observamos que

Age_Sklearn sí cumplía asunción de normalidad
Belly y hdlngth eran independientes, pero no homocedásticas
Chest era independiente y homocedástica

Comenzamos a probar 

In [3]:
lm = ols('age_Sklearn ~ belly + hdlngth' , data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
belly,1.0,0.612830,0.612830,12.920654,0.000507
hdlngth,1.0,0.142017,0.142017,2.994224,0.086647
Residual,100.0,4.743029,0.047430,NaN,NaN


En este caso, la variable belly nos da un valor < 0,05

Vamos a hacer una segunda prueba con variable respuesta age_raiz y chest que era independiente y homocedástica

In [4]:
lm = ols('age_Sklearn ~ chest' , data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
chest,1.0,0.621473,0.621473,12.871936,0.000516
Residual,101.0,4.876403,0.048281,NaN,NaN


Ahora vamos a hacer una tercera prueba con nuevas variables predictoras, no las que habíamos escogido en su momento, para ello volvemos a cargar el dataset de normalización donde age sí estaba normalizada, pero vamos a estandarizar de nuevo todas las variables y probar a ver si el anova mejora metiendo otras variables diferentes

In [5]:
df = pd.read_csv("Datos/possum_norm.csv", index_col = 0)
df.head(2)

,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,age_norm,age_log,age_raiz,age_Sklearn
case,,,,,,,,,,,,,,,,,
1,1,Vic,m,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0,0.522837,2.079442,2.828427,0.625
2,1,Vic,f,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0,0.272837,1.791759,2.449490,0.625


In [6]:
numericas = df.select_dtypes(include = np.number)
numericas.head()

,site,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,age_norm,age_log,age_raiz,age_Sklearn
case,,,,,,,,,,,,,,,
1,1,8.0,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0,0.522837,2.079442,2.828427,0.625
2,1,6.0,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0,0.272837,1.791759,2.449490,0.625
3,1,6.0,94.0,60.0,95.5,39.0,75.4,51.9,15.5,30.0,34.0,0.272837,1.791759,2.449490,0.625
4,1,6.0,93.2,57.1,92.0,38.0,76.1,52.2,15.2,28.0,34.0,0.272837,1.791759,2.449490,0.125
5,1,2.0,91.5,56.3,85.5,36.0,71.0,53.2,15.1,28.5,33.0,-0.227163,0.693147,1.414214,0.000


In [7]:
numericas.drop(['age',"age_norm", "age_log","age_raiz","age_Sklearn", "site" ], axis = 1, inplace = True)

numericas.head(2)

/Users/luciacernuda/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
case,,,,,,,,,
1,94.1,60.4,89.0,36.0,74.5,54.5,15.2,28.0,36.0
2,92.5,57.6,91.5,36.5,72.5,51.2,16.0,28.5,33.0


In [8]:
scaler = StandardScaler()
scaler.fit(numericas)
X_escaladas = scaler.transform(numericas)
numericas_estandar = pd.DataFrame(X_escaladas, columns = numericas.columns)
numericas_estandar.head()

,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly
0,0.420996,1.134883,0.445603,-0.517732,1.381091,1.557431,0.147177,0.491222,1.241874
1,-0.028932,0.231197,1.028384,-0.261331,0.923835,0.750501,0.912499,0.736833,0.150424
2,0.392875,1.005785,1.960834,1.020671,1.586856,0.921668,0.434173,1.473667,0.514241
3,0.167912,0.069825,1.144941,0.507870,1.746896,0.995025,0.147177,0.491222,0.514241
4,-0.310136,-0.188371,-0.370290,-0.517732,0.580893,1.239549,0.051512,0.736833,0.150424


In [10]:
df[numericas_estandar.columns] = numericas_estandar
df.head()

,site,Pop,sex,age,hdlngth,skullw,totlngth,taill,footlgth,earconch,eye,chest,belly,age_norm,age_log,age_raiz,age_Sklearn
case,,,,,,,,,,,,,,,,,
1,1,Vic,m,8.0,-0.028932,0.231197,1.028384,-0.261331,0.923835,0.750501,0.912499,0.736833,0.150424,0.522837,2.079442,2.828427,0.625
2,1,Vic,f,6.0,0.392875,1.005785,1.960834,1.020671,1.586856,0.921668,0.434173,1.473667,0.514241,0.272837,1.791759,2.449490,0.625
3,1,Vic,f,6.0,0.167912,0.069825,1.144941,0.507870,1.746896,0.995025,0.147177,0.491222,0.514241,0.272837,1.791759,2.449490,0.625
4,1,Vic,f,6.0,-0.310136,-0.188371,-0.370290,-0.517732,0.580893,1.239549,0.051512,0.736833,0.150424,0.272837,1.791759,2.449490,0.125
5,1,Vic,f,2.0,0.139791,-0.672489,0.795272,-0.774132,1.083874,1.337359,-0.809475,1.473667,-0.213392,-0.227163,0.693147,1.414214,0.000


In [11]:
lm = ols('age_Sklearn ~ hdlngth + skullw + totlngth + taill + footlgth + earconch + eye + chest + belly' , data=df).fit()
sm.stats.anova_lm(lm)

,df,sum_sq,mean_sq,F,PR(>F)
hdlngth,1.0,0.600484,0.600484,12.417112,0.000665
skullw,1.0,0.015414,0.015414,0.318748,0.573735
totlngth,1.0,0.015928,0.015928,0.329365,0.567434
taill,1.0,0.001674,0.001674,0.034623,0.852797
footlgth,1.0,0.015546,0.015546,0.321472,0.572105
earconch,1.0,0.024961,0.024961,0.516166,0.474302
eye,1.0,0.083086,0.083086,1.718093,0.193201
chest,1.0,0.185834,0.185834,3.842776,0.052986
belly,1.0,0.082274,0.082274,1.701300,0.195372
Residual,92.0,4.449063,0.048359,NaN,NaN
